In [5]:
import pandas as pd
import json
import numpy as np
from collections import OrderedDict,namedtuple
import cudf
import folium
import shapely
from geopy.geocoders import Nominatim
import os
from tqdm import tqdm
import geopandas as gpd
from datetime import datetime
import warnings

In [2]:
REGEN_CSVS=False
def save_csv(ds:pd.DataFrame,name:str,subfolder:str ='',save=REGEN_CSVS):
    if save:
        ds.to_csv(os.path.join('./outputs/',subfolder,f'{name}.csv'))
if REGEN_CSVS:
    print('data is going to be written to disk')
else:
    print('data is not going to be written to disk')

data is not going to be written to disk


In [ ]:
df=pd.DataFrame()
district_counts=[]
for root,dirs,files in os.walk('./data'):
    with tqdm(files) as progbar:
        for i in files:
            ddf=pd.read_csv(root+'/'+i)
            df=pd.concat((df,ddf))
            district_counts.append((i[70:-4],len(ddf),i[70:-4].split('-')[0],i[70:-4].split('-')[1],))
            progbar.set_description(f'read {i[70:]}')
            progbar.update(1)
    save_csv(pd.DataFrame(district_counts,columns=['FQ_District','MSME_Count','District','State-Code']),'district_msme_counts')
    save_csv(df,'combined_data')
df

  0%|          | 0/37 [00:00<?, ?it/s]

read Kasargod-KL.csv: 100%|██████████| 37/37 [00:13<00:00,  2.69it/s]               


,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,33,TAMIL NADU,568,CHENNAI,600040.0,22/12/2021,Global Wings Manpower Service,"137A, Shanthi Colony, First Street, 137A, AI, ...","[{""NIC5DigitId"":""79110"",""Description"":""Travel ..."
1,33,TAMIL NADU,568,CHENNAI,600003.0,08/02/2022,RISHABH EXIM,"No.16/17 SHOP NO 18, Na, Ekambareshwar koil a...","[{""NIC5DigitId"":""47990"",""Description"":""Other r..."
2,33,TAMIL NADU,568,CHENNAI,600017.0,08/02/2022,AQUAFINE,"15/5, AQUAFINE, RADHAKRISHNAN STREET, 15/5, T ...","[{""NIC5DigitId"":""95221"",""Description"":""Repair ..."
3,33,TAMIL NADU,568,CHENNAI,600100.0,30/09/2021,LAKSHMI STORE,"No 61, Ezhil nagar, Main Street, No 61, 14, Ch...","[{""NIC5DigitId"":""47110"",""Description"":""Retail ..."
4,33,TAMIL NADU,568,CHENNAI,600077.0,08/02/2022,SRI LAKSHMI ELECTRICAL SERVICE,"2228, TNHB, AYAPPAKKAM, 2228, TNHB, CHENNAI","[{""NIC5DigitId"":""43122"",""Description"":""Site pr..."
...,...,...,...,...,...,...,...,...,...
25616,32,KERALA,558,KASARAGOD,671542.0,22/02/2025,SATHEESAN T,"KOTTAMKUZHY HOUSE, KOTTAMKUZHY HOUSE, KARADKA,...","[{""NIC5DigitId"":""49224"",""Description"":""Taxi op..."
25617,32,KERALA,558,KASARAGOD,671531.0,22/02/2025,SUNIL KUMAR K,"KALICHAMARAM, KOVVAL STORE, KANHANGAD SOUTH, K...","[{""NIC5DigitId"":""49110"",""Description"":""Passeng..."
25618,32,KERALA,558,KASARAGOD,671312.0,22/02/2025,SHYJESH K V,"1, KOVVAL VEEDU, MAVILAKADAPURAM, 1, NEELESWAR...","[{""NIC5DigitId"":""79110"",""Description"":""Travel ..."
25619,32,KERALA,558,KASARAGOD,671323.0,22/02/2025,BADRIYA KUDUMBASHREE,"00, MAJIBAIL KATTE, MAJIBAIL KATTE, 00, MANJES...","[{""NIC5DigitId"":""17014"",""Description"":""Manufac..."


In [4]:
act_map=lambda x: json.loads(str(x))
df['Activities']=df['Activities'].map(act_map,na_action='ignore')
df['Pincode']=df['Pincode'].convert_dtypes()
df['RegistrationDate']=pd.to_datetime(df['RegistrationDate'],dayfirst=True,errors='coerce')
df

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,33,TAMIL NADU,568,CHENNAI,600040,2021-12-22,Global Wings Manpower Service,"137A, Shanthi Colony, First Street, 137A, AI, ...","[{'NIC5DigitId': '79110', 'Description': 'Trav..."
1,33,TAMIL NADU,568,CHENNAI,600003,2022-02-08,RISHABH EXIM,"No.16/17 SHOP NO 18, Na, Ekambareshwar koil a...","[{'NIC5DigitId': '47990', 'Description': 'Othe..."
2,33,TAMIL NADU,568,CHENNAI,600017,2022-02-08,AQUAFINE,"15/5, AQUAFINE, RADHAKRISHNAN STREET, 15/5, T ...","[{'NIC5DigitId': '95221', 'Description': 'Repa..."
3,33,TAMIL NADU,568,CHENNAI,600100,2021-09-30,LAKSHMI STORE,"No 61, Ezhil nagar, Main Street, No 61, 14, Ch...","[{'NIC5DigitId': '47110', 'Description': 'Reta..."
4,33,TAMIL NADU,568,CHENNAI,600077,2022-02-08,SRI LAKSHMI ELECTRICAL SERVICE,"2228, TNHB, AYAPPAKKAM, 2228, TNHB, CHENNAI","[{'NIC5DigitId': '43122', 'Description': 'Site..."
...,...,...,...,...,...,...,...,...,...
25616,32,KERALA,558,KASARAGOD,671542,2025-02-22,SATHEESAN T,"KOTTAMKUZHY HOUSE, KOTTAMKUZHY HOUSE, KARADKA,...","[{'NIC5DigitId': '49224', 'Description': 'Taxi..."
25617,32,KERALA,558,KASARAGOD,671531,2025-02-22,SUNIL KUMAR K,"KALICHAMARAM, KOVVAL STORE, KANHANGAD SOUTH, K...","[{'NIC5DigitId': '49110', 'Description': 'Pass..."
25618,32,KERALA,558,KASARAGOD,671312,2025-02-22,SHYJESH K V,"1, KOVVAL VEEDU, MAVILAKADAPURAM, 1, NEELESWAR...","[{'NIC5DigitId': '79110', 'Description': 'Trav..."
25619,32,KERALA,558,KASARAGOD,671323,2025-02-22,BADRIYA KUDUMBASHREE,"00, MAJIBAIL KATTE, MAJIBAIL KATTE, 00, MANJES...","[{'NIC5DigitId': '17014', 'Description': 'Manu..."


In [5]:
NIC5DigitId_map={}
for i in enumerate(df['Activities']):
    if i[1] is not np.nan:
        for j in i[1]:
            if j['NIC5DigitId'] in NIC5DigitId_map.keys():
                continue
            else:
                NIC5DigitId_map[j['NIC5DigitId']]=j['Description']
NIC5DigitId_map=OrderedDict(sorted(NIC5DigitId_map.items()))
print(len(NIC5DigitId_map))
NIC5DigitId_map

1177


OrderedDict([('01462', 'Production of eggs'),
             ('01463', 'Operation of poultry hatcheries'),
             ('01492', 'Bee-keeping and production of honey and beeswax'),
             ('01493',
              'Raising of silk worms, production of silk worm cocoons'),
             ('01612', 'Operation of agricultural irrigation equipment'),
             ('01620', 'Support activities for animal production'),
             ('01631',
              'Preparation of crops for primary markets, i.e. cleaning, trimming, grading, disinfecting'),
             ('01632', 'Cotton ginning, cleaning and bailing'),
             ('01633', 'Preparation of tobacco leaves'),
             ('01639', 'Other post harvest crop activities, n.e.c.'),
             ('01640', 'Seed processing for propagation'),
             ('05101', 'Opencast mining of hard coal'),
             ('05102', 'Belowground mining of hard coal'),
             ('05103',
              'Cleaning, sizing, grading, pulverizing, compressi

In [6]:
act_id={}
with tqdm(df['Activities']) as progbar:
    for i in enumerate(df['Activities']):
        if i[1] is not np.nan:
            act_id[i[0]]=[list(j.values())[0] for j in i[1]]
        progbar.update()
df['Activities']=pd.Series(act_id,index=act_id.keys())
df

100%|██████████| 3249263/3249263 [00:04<00:00, 686271.37it/s] 


,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,33,TAMIL NADU,568,CHENNAI,600040,2021-12-22,Global Wings Manpower Service,"137A, Shanthi Colony, First Street, 137A, AI, ...",[79110]
1,33,TAMIL NADU,568,CHENNAI,600003,2022-02-08,RISHABH EXIM,"No.16/17 SHOP NO 18, Na, Ekambareshwar koil a...",[47990]
2,33,TAMIL NADU,568,CHENNAI,600017,2022-02-08,AQUAFINE,"15/5, AQUAFINE, RADHAKRISHNAN STREET, 15/5, T ...",[95221]
3,33,TAMIL NADU,568,CHENNAI,600100,2021-09-30,LAKSHMI STORE,"No 61, Ezhil nagar, Main Street, No 61, 14, Ch...",[47110]
4,33,TAMIL NADU,568,CHENNAI,600077,2022-02-08,SRI LAKSHMI ELECTRICAL SERVICE,"2228, TNHB, AYAPPAKKAM, 2228, TNHB, CHENNAI","[43122, 43211, 43213, 43214, 43219, 33122, 351..."
...,...,...,...,...,...,...,...,...,...
25616,32,KERALA,558,KASARAGOD,671542,2025-02-22,SATHEESAN T,"KOTTAMKUZHY HOUSE, KOTTAMKUZHY HOUSE, KARADKA,...",[56291]
25617,32,KERALA,558,KASARAGOD,671531,2025-02-22,SUNIL KUMAR K,"KALICHAMARAM, KOVVAL STORE, KANHANGAD SOUTH, K...",[71100]
25618,32,KERALA,558,KASARAGOD,671312,2025-02-22,SHYJESH K V,"1, KOVVAL VEEDU, MAVILAKADAPURAM, 1, NEELESWAR...",[46699]
25619,32,KERALA,558,KASARAGOD,671323,2025-02-22,BADRIYA KUDUMBASHREE,"00, MAJIBAIL KATTE, MAJIBAIL KATTE, 00, MANJES...",[56302]


In [7]:
def identify_msme(ds:pd.DataFrame=df,
                  pincode:str=None,
                  district:str=None,
                  regdate:str | datetime=None,
                  activity_id:str=None,
                  state:str=None,
                  bname:str=None,
                  addr:str=None
                  ):
    warnings.simplefilter(action='ignore')
    if state is not None:
        ds=ds.loc[ds['State']==state]
    if district is not None:
        ds=ds.loc[ds['District']==district]
    if pincode is not None:
        ds=ds.loc[ds['Pincode']==pincode]
    if activity_id is not None:
        ds=ds.loc[ds['Activities'].apply(lambda x: activity_id in x if x is not np.nan else False)]
    if regdate is not None:
        ds=ds.loc[ds['RegistrationDate']<=regdate]
    if bname is not None:
        ds=ds[ds['EnterpriseName'].map(lambda x : bname in x if type(x) is str else False)]
    if addr is not None:
        ds=ds[ds['CommunicationAddress'].map(lambda x : addr in x if type(x) is str else False)]   
    return ds

In [8]:
identify_msme(activity_id='64990',pincode=560062)

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
3929,29,KARNATAKA,525,BENGALURU (URBAN),560062,2022-01-27,HI -ALT EXPERT INDIA,"8th main, Part B and 1st , 2nd and 3rd Floor, ...",[64990]
9041,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-01-27,GIRIMAJI EQUIPMENTS,"108/3, Thimmaiah Ind Estate, Kanakapura Road, ...",[64990]
13423,29,KARNATAKA,525,BENGALURU (URBAN),560062,2023-02-25,Gowramma,"10/7, Near srindhi school, Srindhi layout road...",[64990]
26025,29,KARNATAKA,525,BENGALURU (URBAN),560062,2022-10-12,ST Infrastructure,"No.991, Manavarthekaval, Chunchanaghatta Main ...","[63999, 64199, 64990, 66309]"
27277,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-07-29,B P Shivabasappa,"No.48,, -, 3RD CROSS GOKULA NAGAR, No.48,, -, ...",[64990]
...,...,...,...,...,...,...,...,...,...
62042,29,KARNATAKA,526,BENGALURU (RURAL),560062,2021-06-11,Rajesh enterprise,"1, Nilaya, Konankunte, 1, 1, Bengaluru","[46599, 25933, 28180, 52291, 64990, 28221]"
63903,29,KARNATAKA,526,BENGALURU (RURAL),560062,2023-06-18,Bismilla store,"#653, Bismilla store, Near aqsa masjid, #653, ...","[64990, 69201, 69202]"
64449,29,KARNATAKA,526,BENGALURU (RURAL),560062,2023-06-08,Kanhaiya infrastructure,"32, Kanhaiya infrastructure, Kanakapura main r...",[64990]
65324,29,KARNATAKA,526,BENGALURU (RURAL),560062,2020-07-28,S V PIPES,"NO,85, GROUND FLOOR, KENGERI HOBLI, NO,85, KEN...",[64990]


In [9]:
identify_msme(addr='vajarahalli')

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
26810,29,KARNATAKA,525,BENGALURU (URBAN),560062,2022-10-30,MEZHA PRASAD ENGINEERS,"Ground floor,, NA, mantapa, vajarahalli, Grou...",[71100]
54202,29,KARNATAKA,525,BENGALURU (URBAN),560062,2023-11-21,sri honnamma auto diesel works,"27, vajarahalli, kanakapura main road, 27, 1, ...",[96098]
165965,29,KARNATAKA,525,BENGALURU (URBAN),560062,2024-03-25,MAMATHA ENTERPRISES,"No 2073 BCCHA LAYOUT, RAGHUVANAHALLI, Near vaj...",[10799]
268550,29,KARNATAKA,525,BENGALURU (URBAN),560062,2024-07-24,venkateshwara dry cleaner,"1, vajarahalli, vajarahalli bus stop, 1, 1, Ba...",[46909]
275566,29,KARNATAKA,525,BENGALURU (URBAN),560062,2025-01-31,Bhuvana Electricals,"NO.78/7, 11th Cross, 7th main road, vajarahall...","[49231, 36000]"
305376,29,KARNATAKA,525,BENGALURU (URBAN),560083,2023-09-22,sri sai granites,"52, junglepalya gate bus stand, BANGALORE, 52,...",[95120]
305600,29,KARNATAKA,525,BENGALURU (URBAN),560062,2023-04-08,CHETAN J,"282, vajarahalli, THALAGHATTAPURA, 282, THALAG...",[10306]
308303,29,KARNATAKA,525,BENGALURU (URBAN),560062,2022-02-21,SHARP SALES CORPORATION,"no 31, nansu vajarahalli talgatpura, kanakpura...",[55101]
343056,29,KARNATAKA,525,BENGALURU (URBAN),560062,2023-11-03,PARI WATER SOLUTIONS,"F5, 1 Floor, A Block, shivaganga galaxy, kanak...",[96010]
407942,29,KARNATAKA,525,BENGALURU (URBAN),560109,2023-01-16,ss electricals,"39 ground floor,4th, nanjundeswara layout, tha...",[56291]


In [10]:
pin_map1={}
temp_df1,temp_df2=(identify_msme(district='BENGALURU (RURAL)'),identify_msme(district='BENGALURU (URBAN)'))
temp_df=pd.concat((temp_df1,temp_df2))
with tqdm(temp_df['Pincode'].unique()) as progbar:
    for i in temp_df['Pincode'].unique():
        pin_map1[i]=(
            len(identify_msme(temp_df1,i)),
            len(identify_msme(temp_df2,i)),
            )
        progbar.update()
if pd.NA in pin_map1.keys():
    pin_map1.pop(pd.NA)
blore_pins=pd.DataFrame(pin_map1,index=['RURAL','URBAN']).T.reset_index().rename(lambda x: 'pincode' if x=='index' else x,axis=1)
save_csv(blore_pins,'blore_pin_msme')

100%|██████████| 1700/1700 [00:01<00:00, 1221.38it/s]


In [11]:
pin_map={}
gpu_df=cudf.from_pandas(df['Pincode'])
with tqdm(gpu_df.unique()) as progbar:
    for i in gpu_df.unique().to_pandas().convert_dtypes(int):
        pin_map[i]=gpu_df.where(gpu_df==i).dropna().count()
        progbar.update()
if pd.NA in pin_map.keys():
    pin_map.pop(pd.NA)
pin_map=OrderedDict(sorted(pin_map.items(),key=lambda x:x[1],reverse=True))

100%|██████████| 10245/10245 [00:18<00:00, 543.22it/s]


In [12]:
tech_ids={}
for k,v in NIC5DigitId_map.items():
    if 'food' in v: 
        tech_ids[k]=v
tech_ids

{'10202': 'Artificial dehydration of fish and sea food',
 '10203': 'Radiation preservation of fish and similar food',
 '10204': 'Processing and preserving of fish crustacean and similar foods',
 '10502': 'Manufacture of milk-powder, ice-cream powder and condensed milk except baby milk food',
 '10503': 'Manufacture of baby milk foods',
 '10616': 'Manufacture of cereal breakfast foods obtained by roasting or swelling cereal grains',
 '10794': 'Manufacture of malted foods including foods for infants and handicapped',
 '10796': 'Manufacture of papads, appalam and similar food products',
 '10798': 'Processing of salt into food-grade salt, e.g. iodized salt',
 '10799': 'Other semi-processed, processed or instant foods n.e.c. except farinaceous products and malted foods and manufacturing activities like manufacture of egg powder, sambar powder etc. (this excludes the activities covered under 10619)',
 '25992': 'Manufacture of containers such as tins and cans for food products and boxes',
 '28

In [13]:
identify_msme(state='KARNATAKA',addr='doddakallasandra').reset_index(drop=True)

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-07-19,MAHALAKSHMI PACKERS,"No 1113/13 16th main, MAHALAKSHMI PACKERS, dod...",[45401]
1,29,KARNATAKA,525,BENGALURU (URBAN),560062,2024-04-28,RV ENTERPRISES,"18, kirana, ashwath layout, 18, doddakallasand...",[10799]
2,29,KARNATAKA,525,BENGALURU (URBAN),560062,2025-02-07,SQUARE IMAGE INTERIO STUDIO,"1 ST FLOOR NO 470, BOOHBCS 1 st stage narayan ...","[13911, 13913, 13999, 14201]"
3,29,KARNATAKA,525,BENGALURU (URBAN),560062,2024-01-26,S K Apparels,"no 38, sk apparels, 38, Annpurneswari industri...",[47411]
4,29,KARNATAKA,525,BENGALURU (URBAN),560062,2022-09-28,CR AQUA TEC,"14/14/1 government, School road doddakallasand...",[10799]
5,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-11-10,karthik energy solutions,"no 1, doddakallasandra, doddakallasandra, no 1...",[56102]
6,29,KARNATAKA,525,BENGALURU (URBAN),560062,2024-01-25,Girish kumar S,"62, 1st cross, Bangalore, 62, doddakallasandra...","[95120, 95210]"
7,29,KARNATAKA,525,BENGALURU (URBAN),560062,2020-10-21,S/M Ranganatha Provision Stores,"1032 1st cross, 1st main road, doddakallasandr...",[47190]
8,29,KARNATAKA,526,BENGALURU (RURAL),560062,2022-09-27,SUNRISE PAPER PRODUCTS,"No 13/4, doddakallasandra Village, Nararayana ...",[56292]
9,29,KARNATAKA,526,BENGALURU (RURAL),560062,2021-09-21,Anahatha Solution,"Nirmal Farm, konankunte cross, vasanthapura Ma...",[56291]


In [14]:
{96098:NIC5DigitId_map['96098'],56291:NIC5DigitId_map['56291']}

{96098: 'General household maintenance activities like grooming of the floor, dusting, cleaning of utensils etc.',
 56291: 'Activities of food service contractors (e.g. for transportation companies)'}

In [15]:
tdf=identify_msme(pincode=560062,district='BENGALURU (URBAN)',)
identify_msme(identify_msme(tdf,activity_id='96098'),activity_id='56291')

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
78290,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-02-24,MAHESH VIBHU K A PROP MATASHRI CONDIMENTS,"NO 417, NEAR SHOBHA FOREST VIEW APARTMENT, NEA...","[32909, 56291, 10795, 96098]"
280549,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-07-15,M/S JYOTHY INDUSTRIES,"UNIT-1 , 21/10A,, JYOTHY INDUSTRIES, KANAKAPUR...","[56291, 96098]"


In [16]:
identify_msme(district='BENGALURU (URBAN)',activity_id='45200')

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
240,29,KARNATAKA,525,BENGALURU (URBAN),560068,2023-05-12,A M Chicken Center,"No 10, NGR LAYOUT, SRC Complex, Bommanahalli, ...",[45200]
320,29,KARNATAKA,525,BENGALURU (URBAN),560085,2023-05-11,RADHA,"123, BALAJI APARTMENT, ITTAMDU, 123, D BLOCK, ...","[45200, 45403]"
323,29,KARNATAKA,525,BENGALURU (URBAN),560010,2023-09-08,M/s. POWERFULL SOLUTIONS,"NO. 3Y/1,, M/s. POWERFULL SOLUTIONS, 14TH MAIN...","[45200, 45403, 96093]"
575,29,KARNATAKA,525,BENGALURU (URBAN),560032,2022-05-13,M/S G P J H TRAVELOGUE REPS PRIVATE LIMITED,"NO 73, NO 73, R T NAGAR, NO 73, DINNUR, BANGALORE","[45200, 45300, 45402]"
681,29,KARNATAKA,525,BENGALURU (URBAN),562107,2022-07-02,Kusuma Ladies Tailors,"Mallenahalli, Yellemma Devi Temple, Yellemma D...",[45200]
...,...,...,...,...,...,...,...,...,...
465520,29,KARNATAKA,525,BENGALURU (URBAN),560078,2024-12-24,Shivanna Garden Work,"5 ,, Shivana nilaya, 1st cross , 22nd main, 5 ...",[45200]
465680,29,KARNATAKA,525,BENGALURU (URBAN),561203,2024-12-25,THE NEW LESSON FAMILY RESTAURANT,"Survey No -90/1,, Hesarghatta Hobli, Doddaball...",[45200]
466450,29,KARNATAKA,525,BENGALURU (URBAN),560041,2025-01-15,ANNANNYA TRAVELS AND CARGO,"No.281/A, ,, 9TH A MAIN, 40TH CROSS, No.281/A,...",[45200]
467040,29,KARNATAKA,525,BENGALURU (URBAN),560096,2025-01-15,VAISHNAV SURGICALS,"NO.188, SRINIVAS NAGAR, 22ND MAIN ROAD,, NO.18...",[45200]


In [17]:
pd.concat(
    (identify_msme(pincode=560062,district='BENGALURU (URBAN)',activity_id='56102',regdate='2021-12-01'),
     identify_msme(pincode=560062,district='BENGALURU (URBAN)',activity_id='56102',regdate='2021-12-01'))
).reset_index(drop=True).head(50)

,LG_ST_Code,State,LG_DT_Code,District,Pincode,RegistrationDate,EnterpriseName,CommunicationAddress,Activities
0,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-06-16,SRI JANANI TEX PROCESS,"SY NO.59, GNC INDUSTRIAL ESTAT, BWSSB IB ROAD ...","[56102, 56103, 56304, 10750]"
1,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-08-04,SHREE PROPERTIES AND REAL ESTATE,"NO.470,, NARAYAN NAGAR, NEAR KANAKAPURA MAIN R...",[56102]
2,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-07-30,BHAGAVATHI BANASHANKARI BUILDERS,"NO.55/3/2, 2ND FLOOR, CHUNCHAGATTA MAIN ROAD, ...","[56102, 56291]"
3,29,KARNATAKA,525,BENGALURU (URBAN),560062,2020-11-03,Sri Rajarajeshwari Enterprises,"No 48, Sri Giri, B jai Kumar Layout, No 48, Be...",[56102]
4,29,KARNATAKA,525,BENGALURU (URBAN),560062,2020-10-21,Sumanth Enterprises,"#556/102, BCMC Layout, Doddakallasandra, #556/...","[56101, 56102, 10750]"
5,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-03-12,M/S VRESQ SYSTEMS AND SERVICES,"No. 31, 6th Cross, MC Industrial Layout, Chunc...",[56102]
6,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-03-01,VINCENT VENGAGH GARMENTS,"3&4, suprajanagar, Konanakunte post, 3&4, Chun...",[56102]
7,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-02-08,A1 SAFETY FIRE EXTINGUISHER,"11, Ground Floor, Kanakapura Main Road, 11, Ra...",[56102]
8,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-06-18,M RAMESH,"NO.407, 4TH FLOOR,, PYRAMID PLATINA APARTMENTS...",[56102]
9,29,KARNATAKA,525,BENGALURU (URBAN),560062,2021-02-11,ABHAY ENTERPRISES,"NO 359/1, 1ST FLOOR, J P NAGAR 9TH PHASE, NO 3...",[56102]


In [18]:
digit_map={}
D_NAMES=['BENGALURU (RURAL)','BENGALURU (URBAN)','RAMANAGARA']
for d_name in D_NAMES:
    temp_df=identify_msme(district=d_name)
    with tqdm(list(NIC5DigitId_map.keys())) as progbar:
        for i in NIC5DigitId_map.keys():
            digit_map[i]=len(identify_msme(temp_df,activity_id=i))
            progbar.update(1)
    digit_map=OrderedDict(sorted(digit_map.items()))
    s=pd.DataFrame((digit_map.items()),columns=['NIC5digitId',d_name])
    s['Activity']=s['NIC5digitId'].map(lambda x:NIC5DigitId_map[x])
    save_csv(s,f'digit_hm_{d_name}')
    msme_list=pd.DataFrame()
    id_freq=dict()
    for i in tech_ids.keys():
        t=identify_msme(activity_id=i,district=d_name)
        id_freq[i]=len(t)
        msme_list=pd.concat((msme_list,t),)
    msme_list=msme_list.drop_duplicates(subset=msme_list.columns[:-1])
    print(msme_list)
    dsc=pd.DataFrame(id_freq.items(),columns=['NIC5digitId','tech_msme_count'])
    dsc['Activity']=dsc['NIC5digitId'].map(lambda x:NIC5DigitId_map[x])
    save_csv(dsc,f'tech_hm_{d_name}')
    print(f'\nfinished {d_name}\n')

  0%|          | 0/1177 [00:00<?, ?it/s]

100%|██████████| 1177/1177 [00:12<00:00, 95.07it/s] 


       LG_ST_Code      State  LG_DT_Code           District  Pincode  \
4064           29  KARNATAKA         526  BENGALURU (RURAL)   560091   
5332           29  KARNATAKA         526  BENGALURU (RURAL)   560105   
6442           29  KARNATAKA         526  BENGALURU (RURAL)   560105   
8890           29  KARNATAKA         526  BENGALURU (RURAL)   562123   
10289          29  KARNATAKA         526  BENGALURU (RURAL)   562106   
...           ...        ...         ...                ...      ...   
86597          29  KARNATAKA         526  BENGALURU (RURAL)   560017   
86630          29  KARNATAKA         526  BENGALURU (RURAL)   560049   
86634          29  KARNATAKA         526  BENGALURU (RURAL)   562123   
86644          29  KARNATAKA         526  BENGALURU (RURAL)   560049   
86669          29  KARNATAKA         526  BENGALURU (RURAL)   562122   

      RegistrationDate              EnterpriseName  \
4064        2021-07-13                   SHYLAJA R   
5332        2022-05-23  BHU

100%|██████████| 1177/1177 [01:01<00:00, 19.03it/s]


        LG_ST_Code      State  LG_DT_Code           District  Pincode  \
4064            29  KARNATAKA         525  BENGALURU (URBAN)   560085   
5332            29  KARNATAKA         525  BENGALURU (URBAN)   560102   
6442            29  KARNATAKA         525  BENGALURU (URBAN)   560074   
8890            29  KARNATAKA         525  BENGALURU (URBAN)   560064   
10289           29  KARNATAKA         525  BENGALURU (URBAN)   560062   
...            ...        ...         ...                ...      ...   
467197          29  KARNATAKA         525  BENGALURU (URBAN)   560066   
467282          29  KARNATAKA         525  BENGALURU (URBAN)   560072   
467368          29  KARNATAKA         525  BENGALURU (URBAN)   560077   
467409          29  KARNATAKA         525  BENGALURU (URBAN)   560049   
467457          29  KARNATAKA         525  BENGALURU (URBAN)   560037   

       RegistrationDate                                     EnterpriseName  \
4064         2022-03-29                      

100%|██████████| 1177/1177 [00:04<00:00, 294.17it/s]


       LG_ST_Code      State  LG_DT_Code    District  Pincode  \
4064           29  KARNATAKA         631  RAMANAGARA   562159   
5332           29  KARNATAKA         631  RAMANAGARA   562138   
6442           29  KARNATAKA         631  RAMANAGARA   562160   
8890           29  KARNATAKA         631  RAMANAGARA   571404   
10289          29  KARNATAKA         631  RAMANAGARA   562159   
...           ...        ...         ...         ...      ...   
30733          29  KARNATAKA         631  RAMANAGARA   562159   
30735          29  KARNATAKA         631  RAMANAGARA   562117   
30851          29  KARNATAKA         631  RAMANAGARA   562159   
30882          29  KARNATAKA         631  RAMANAGARA   562159   
30973          29  KARNATAKA         631  RAMANAGARA   562117   

      RegistrationDate          EnterpriseName  \
4064        2023-06-19                  RESHMA   
5332        2024-08-01                VIJENDRA   
6442        2022-07-15              MOIN PRESS   
8890        2023-12

In [19]:
tech_district_msme=pd.DataFrame()
with tqdm(df['District'].unique()) as progbar:
    for i in df['District'].unique():
        temp_df=identify_msme(district=i)
        t=[]
        for j in tech_ids.keys():
            t.append(len(identify_msme(temp_df,activity_id=j)))
        tech_district_msme=pd.concat((tech_district_msme,pd.DataFrame(t,columns=[i],index=tech_ids)),axis=1)
        progbar.update()

100%|██████████| 37/37 [00:13<00:00,  2.72it/s]


In [21]:
class CustomNominatim(Nominatim):
    def query(self, address, exactly_one=False, timeout=10):
        # Override the scheme from 'https' to 'http'
        url = self._url.format(scheme='http', **self._query_params)
        return super(CustomNominatim, self).query(address, exactly_one, timeout)

geolocator = CustomNominatim(domain='localhost:8080',user_agent="pincode_mapper",scheme='http')
locations=[]
with tqdm((pin_map.keys()),desc='retriving location data from local nominatim server') as progbar:
    for i in pin_map.keys():
        locations.append(geolocator.geocode(i,exactly_one=True,country_codes=['in'],geometry='geojson'))
        progbar.update()
len(locations),len(pin_map)

retriving location data from local nominatim server:   0%|          | 0/10244 [00:00<?, ?it/s]

retriving location data from local nominatim server: 100%|██████████| 10244/10244 [00:33<00:00, 305.62it/s]


(10244, 10244)

In [22]:
extra_districts=gpd.GeoDataFrame()
for (froot,dirs,files) in os.walk('./PincodeBoundary'):
    for i in files:
        if i.endswith('.geojson') and 'Bangalore' not in froot:
            # print(os.path.join(froot,i))
            f=gpd.read_file(os.path.join(froot,i))
            # print(f.columns)
            if 'pin_code' not in f:
                f['pin_code']=f['pin']
            f=f[['pin_code','geometry','area_name']]
            extra_districts=pd.concat((extra_districts,f)).reset_index(drop=True)
extra_districts=extra_districts.T[extra_districts['pin_code'].dropna().index].T
extra_districts=extra_districts.T[extra_districts['geometry'].dropna().index].T
extra_districts.reset_index(drop=True,inplace=True)
extra_districts['pin_code']=extra_districts['pin_code'].convert_dtypes(int)
extra_districts=(extra_districts[['pin_code','area_name','geometry']].set_index('pin_code')).to_dict()

for i in extra_districts['area_name'].keys():
    extra_districts[i]=(extra_districts['area_name'][i],shapely.geometry.mapping(extra_districts['geometry'][i]))#json.dumps
extra_districts.pop('area_name')
extra_districts.pop('geometry')
extra_districts[list(extra_districts.keys())[0]]

('GOREGAON EAST',
 {'type': 'Polygon',
  'coordinates': (((72.85856454699197, 19.177806168902624),
    (72.85861248109552, 19.17760036326207),
    (72.85867572877973, 19.177377291116652),
    (72.8587408961637, 19.177200245368343),
    (72.8588299281915, 19.17692812368156),
    (72.85892633089622, 19.176669470126118),
    (72.85901278386083, 19.176402336562326),
    (72.85910110442298, 19.176185173239936),
    (72.85925404709165, 19.175638484157563),
    (72.85931281667416, 19.175372460423887),
    (72.85936657635237, 19.175104945488332),
    (72.85940022634458, 19.174836478126796),
    (72.85941874152091, 19.174571419515516),
    (72.8594133588157, 19.174404066328876),
    (72.85959361925221, 19.17444016913274),
    (72.85959675077929, 19.174440788522713),
    (72.86085576031124, 19.17469293859836),
    (72.8612299050338, 19.174767867685148),
    (72.86153204888804, 19.17482901407103),
    (72.8632815367685, 19.175183057868246),
    (72.86398384864076, 19.1753251800167),
    (72.86568

In [23]:
valid_locs={}
with tqdm(locations) as progbar:
    for i in enumerate(locations):
        if i[1] and i[1].raw['name'].isnumeric() and int(i[1].raw['name'])>100000:
            if i[1].raw['geojson']['type'] =='Polygon':
                valid_locs[int(i[1].raw['name'])]=(i[1].raw['display_name'],i[1].raw['geojson'])
            elif int(i[1].raw['name']) in np.array(list(extra_districts.keys()),dtype=int):
                valid_locs[int(i[1].raw['name'])]=extra_districts[int(i[1].raw['name'])]
        progbar.update()
print(f'{len(valid_locs)} valid location geometries identified for plotting out of {len(locations)}')

100%|██████████| 10244/10244 [00:00<00:00, 139282.14it/s]

477 valid location geometries identified for plotting out of 10244


In [24]:
def process_geojson(pin,valid_locs=valid_locs, count_mapper=pin_map):
    feature = {
        "type": "Feature",
        "properties": {
            "name": pin,
            "counts":int(count_mapper.get(pin, 0)),  
            "address":valid_locs[pin][0]
        },
        "geometry": valid_locs[pin][1]  
    }

    geojson = {
        "type": "FeatureCollection",
        "features": [feature]
    }

    return geojson

some map regions do not have pincode boundary data in nominatim and in the data taken from the [PincodeBoundary](https://github.com/datameet/PincodeBoundary) repo, removing them from being plotted on the map.

In [ ]:
x=folium.Map(location=[12.994702199999999, 77.4849282381972], zoom_start=8)
with tqdm(valid_locs.keys()) as progbar:
      for i in valid_locs.keys():
            c=hex(int(255*pin_map[i]/list(pin_map.values())[0]))
            folium.Choropleth(
                  geo_data=(valid_locs[i][1]),
                  highlight=True,
                  fill_color=f"#{str(c)[2:]}ffff",
                  line_color='#000000',
                  ).add_to(x)
            progbar.update()
with tqdm(valid_locs.keys()) as progbar:
      for i in valid_locs.keys():
            folium.GeoJson(
                  process_geojson(i),
                  name="labels",
                  tooltip=folium.GeoJsonTooltip(
                        fields=["name",'counts','address'],
                        aliases=["Pincode",'MSME counts','pincode address'],
                        labels=True,
                        sticky=False
                  )
            ).add_to(x)
            progbar.update()
x.save('./outputs/map.html')
x

GADM data

In [6]:
ind_data=gpd.read_file('./gadm_pin_ind',layer=0)


In [ ]:
x=folium.Map(location=[12.994702199999999, 77.4849282381972], zoom_start=7)
folium.GeoJson(ind_data).add_to(x)
x

In [8]:
x.save('./outputs/india_gadm.html')